<a href="https://colab.research.google.com/github/pjbenard/MPEG/blob/main/JPEG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import scipy as sp
import scipy.fftpack as fft

# from bokeh.plotting import figure, show
# from bokeh.io import output_notebook
# import holoviews as hv
# hv.config.enable_colab_support = True
# hv.extension('bokeh')

import matplotlib.pyplot as plt

from PIL import Image
import requests
from io import BytesIO

In [ ]:
img_url = "https://upload.wikimedia.org/wikipedia/commons/2/28/RGB_illumination.jpg"
response = requests.get(img_url)
img = np.array(Image.open(BytesIO(response.content))).astype(int)

In [ ]:
img.shape

In [ ]:
def plot_img_channels(img, cmaps=['Reds', 'Greens', 'Blues']):
  fig, axs = plt.subplots(1, 4, figsize=(16, 3), sharey=True, sharex=True)
  axs[0].imshow(img.astype(int))
  
  for col, cmap in enumerate(cmaps):
    axs[col + 1].imshow(img[...,col], cmap=cmap)

  plt.show()

plot_img_channels(img)

In [ ]:
def RGB_to_YCbCr(img_rgb):
    conv = np.array([[ 65.481, 128.553,  24.966], 
                     [-37.797, -74.203, 112.   ], 
                     [112.   , -93.786, -18.214]])
    
    img_ycbcr = np.dot(img_rgb.astype(float)/255, conv.T)
    img_ycbcr[:,:,0] += 16
    img_ycbcr[:,:,[1,2]] += 128
    return img_ycbcr.astype(int)


def YCbCr_to_RGB(img_ycbcr):
    conv = np.array([[1,  0      , 1.402  ], 
                     [1, -0.34414, -.71414], 
                     [1,  1.772  , 0      ]])

    img_rgb = img_ycbcr.astype(float)
    img_rgb[:,:,[1,2]] -= 128
    img_rgb = np.dot(img_rgb, conv.T)
    
    return np.clip(img_rgb, 0, 255).astype(int)

In [ ]:
img_ycbcr = RGB_to_YCbCr(img)
print(img_ycbcr.shape)
plot_img_channels(img_ycbcr, ['gray'] * 3)

In [ ]:
plot_img_channels(YCbCr_to_RGB(img_ycbcr))

In [ ]:
img_yuv = Image.open(BytesIO(response.content)).convert('YCbCr')

plot_img_channels(np.array(img_yuv), ['gray'] * 3)

In [ ]:
plt.imshow(np.sum(img, axis=-1), cmap='gray')

In [ ]:
def shift_array(arr, shift=-128):
    return arr + shift

In [ ]:
def transform_into_blocks(img, block_size=8):
    """
    Return a array of size (img.shape[0] // block_size * img.shape[1] // block_size, 3, block_size, block_size) or
                         (3, img.shape[0] // block_size * img.shape[1] // block_size, block_size, block_size) (TBD)
    First shape reads block from top to bottom, from left to right.
    """
    nb_blocks_height = img.shape[0] // block_size
    nb_blocks_width  = img.shape[1] // block_size

    blocks = np.empty((nb_blocks_height, nb_blocks_width, 3, block_size, block_size), dtype=img.dtype)

    for y in range(nb_blocks_height):
        for x in range(nb_blocks_width):
            for color in range(3):
                blocks[y, x, color] = img[y * block_size:(y + 1) * block_size, 
                                          x * block_size:(x + 1) * block_size, 
                                          color]

    return blocks

In [ ]:
blocks = transform_into_blocks(img_ycbcr)

In [ ]:
blocks.shape

IMG = Y, U, V

In [ ]:
b1 = shift_array(blocks[0, 0, 1])
b1

In [ ]:
dct1 = fft.dctn(b1)
dct1.shape, dct1.astype(int)

In [ ]:
dct_tot = fft.dctn(shift_array(blocks), axes=[-2, -1])
dct_tot.shape

In [ ]:
dct_tot[0, 0, 1].astype(int)

In [ ]:
def apply_dct(blocks):
    return fft.dctn(blocks, axes=[-2, -1])

In [ ]:
blocks_dct = apply_dct(shift_array(blocks))

In [ ]:
quantization_matrix = np.array([[16, 11, 10, 16,  24,  40,  51,  61],
                                [12, 12, 14, 19,  26,  58,  60,  55],
                                [14, 13, 16, 24,  40,  57,  69,  56],
                                [14, 17, 22, 29,  51,  87,  80,  62], 
                                [18, 22, 37, 56,  68, 109, 103,  77], 
                                [24, 35, 55, 64,  81, 104, 113,  92], 
                                [49, 64, 78, 87, 103, 121, 120, 101], 
                                [72, 92, 95, 98, 112, 100, 103,  99]], dtype=int)

In [ ]:
def quantize(arr, quant_mat=quantization_matrix):
    return np.round(np.divide(arr, quant_mat)).astype(int)

def dequantize(arr, quant_mat=quantization_matrix):
    return np.multiply(arr, quant_mat)

In [ ]:
blocks_quant = quantize(blocks_dct)

In [ ]:
blocks_quant[0, 0, 1]

In [ ]:
blocks_dequant = dequantize(blocks_quant)

In [ ]:
def transform_into_image(blocks, block_size=8):
    """
    Return a array of size (img.shape[0] // block_size * img.shape[1] // block_size, 3, block_size, block_size) or
                         (3, img.shape[0] // block_size * img.shape[1] // block_size, block_size, block_size) (TBD)
    First shape reads block from top to bottom, from left to right.
    """
    img_height = blocks.shape[0] * block_size
    img_width  = blocks.shape[1] * block_size

    img = np.empty((img_height, img_width, 3), dtype=blocks.dtype)

    for i in range(blocks.shape[0]):
        for j in range(blocks.shape[1]):
            for color in range(3):
                img[
                    i * block_size : (i + 1) * block_size, 
                    j * block_size : (j + 1) * block_size, 
                    color,
                ] = blocks[i, j, color]

    return img

In [ ]:
blocks.shape

In [ ]:
img_deblocked = transform_into_image(blocks_dct)
img_deblocked.shape

In [ ]:
plot_img_channels(img_deblocked, ['gray'] * 3)